In [6]:
import os
from time import sleep
import time
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import csv

In [1]:
!pip install webdriver-manager

You should consider upgrading via the 'C:\Users\tinak\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\tinak\AppData\Local\Temp/ipykernel_9984/3871533204.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [8]:
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="a6d1fc7eccd2ef5f7fcd4a9a127256f6")>

In [15]:
gu_list = ['충북']

In [16]:
for index, gu_name in enumerate(gu_list):
    fileName = '충북동물병원.csv'
    file = open(fileName, 'w', encoding = 'utf-8-sig')
    file.write("동물병원명" + "|" "주소" + "|" + "지번" + "|" + "영업시간" + "|" + "전화번호" + "|" + "평점"+ "|"\
               + "리뷰수" + "|" + "링크" +"\n")
    file.close()

    options = webdriver.ChromeOptions()
    # options.add_argument('headless')
    options.add_argument('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36')
    options.add_argument('lang=ko_KR')
    chromedriver_path = "./chromedriver.exe"
    driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기
    driver.get('https://map.kakao.com/')  # 주소 가져오기
    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]') # 검색 창
    search_area.send_keys(gu_name + ' 동물병원')  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    driver.implicitly_wait(3) # 기다려 주자
    more_page = driver.find_element_by_id("info.search.place.more")
    # more_page.click()
    more_page.send_keys(Keys.ENTER) # 더보기 누르고
    # 첫 번째 검색 페이지 끝
    # driver.implicitly_wait(5) # 기다려 주자
    time.sleep(3)

    # next 사용 가능?
    next_btn = driver.find_element_by_id("info.search.page.next")
    has_next = "disabled" not in next_btn.get_attribute("class").split(" ")
    Page = 1
    while has_next: # 다음 페이지가 있으면 loop
    # for i in range(2, 6): # 2, 3, 4, 5
        file = open(fileName, 'a', encoding='utf-8-sig')
        time.sleep(1)
        page_links = driver.find_elements(By.CSS_SELECTOR, "#info\.search\.page a")
        pages = [link for link in page_links if "HIDDEN" not in link.get_attribute("class").split(" ")]
        print(len(pages), "개의 페이지 있음")
        # pages를 하나씩 클릭하면서
        for i in range(1, 6):
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            try:
                page = driver.find_element_by_xpath(xPath)
                page.send_keys(Keys.ENTER)
            except ElementNotInteractableException:
                print('End of Page')
                break;
            time.sleep(3)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            hospital_lists = soup.select('.placelist > .PlaceItem')
            count = 1
            for hospital in hospital_lists:
                #temp = []
                hospital_name = hospital.select('.head_item > .tit_name > .link_name')[0].text
                hospital_addr = hospital.select('div.info_item > div.addr > p')[0].text
                hospital_road_addr = hospital.select('div.info_item > div.addr > p.lot_number')[0].text
                hospital_hour = hospital.select('div.info_item > div.openhour > p > a')[0].text
                hospital_tel = hospital.select('div.info_item > div.contact > span.phone')[0].text
                hospital_score_list = hospital.select('div.rating > span > em.num')
                if hospital_score_list:
                    hospital_score = hospital_score_list[0].text
                else:
                    hospital.score = "N/A"
                hospital_review = hospital.select('div.rating > a.review > em')[0].text
                hospital_link = hospital.select('div.info_item > div.contact > a.moreview')[0]['href']

                file.write(hospital_name + "|" + hospital_addr + "|" + hospital_road_addr + "|" + hospital_hour + "|" + hospital_tel + "|"\
                           + hospital_score + "|" + hospital_review + "|" + hospital_link+ "\n")
            print(i, ' of', ' [ ' , Page, ' ] ')
        next_btn = driver.find_element_by_id("info.search.page.next")
        has_next = "disabled" not in next_btn.get_attribute("class").split(" ")
        if not has_next:
            print('Arrow is Disabled')
            driver.close()
            file.close()
            break # 다음 페이지 없으니까 종료
        else: # 다음 페이지 있으면
            Page += 1
            next_btn.send_keys(Keys.ENTER)

    file.close()

    print("End of Crawl")

C:\Users\tinak\AppData\Local\Temp/ipykernel_9984/784985054.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기
C:\Users\tinak\AppData\Local\Temp/ipykernel_9984/784985054.py:15: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]') # 검색 창
C:\Users\tinak\AppData\Local\Temp/ipykernel_9984/784985054.py:17: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
C:\Users\tinak\AppData\Local\Temp/ipykernel_9984/784985054.py:19: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  more_page = driver.find_element_by_id("info.search.plac

5 개의 페이지 있음


C:\Users\tinak\AppData\Local\Temp/ipykernel_9984/784985054.py:41: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  page = driver.find_element_by_xpath(xPath)


1  of  [  1  ] 
2  of  [  1  ] 
3  of  [  1  ] 
4  of  [  1  ] 
5  of  [  1  ] 


C:\Users\tinak\AppData\Local\Temp/ipykernel_9984/784985054.py:70: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  next_btn = driver.find_element_by_id("info.search.page.next")


5 개의 페이지 있음
1  of  [  2  ] 
2  of  [  2  ] 
3  of  [  2  ] 
4  of  [  2  ] 
5  of  [  2  ] 
Arrow is Disabled
End of Crawl


In [17]:
data = list()
f = open('충북동물병원.csv', 'r', encoding = 'utf-8-sig')
file = csv.reader(f, delimiter = '|')
lines = []
for line in file:
    lines.append(line)

f = open('충북동물병원.csv', 'w', newline = "")
new_file = csv.writer(f)
new_file.writerows(lines)

f.close()